<H1>MAPK8IP3 genotype phenotype correlations</H1>

In [4]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../src'))

from genophenocorr import *

In [6]:
allPatients = Cohort('phenopackets/*.json', ref = 'hg38')

ValueError: Invalid contig name 'chr' for reference 'GRCh38'